# DATA 603 Project
## US COVID-19 Mortality Modelling 

### Imports and Utility

In [144]:
import re

import mysql.connector
import pandas as pd
from mysql.connector import errorcode

In [145]:
# SQL Query Function
# Reference: MySQL Developer's guide. Accessed November 18
# https://dev.mysql.com/doc/connector-python/en/connector-python-example-cursor-select.html
# https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor.html


def run_sql(query):
    df = None
    try:
        cnx = mysql.connector.connect(option_files=['connection.conf', 'password.conf'])
        cur = cnx.cursor()

        cur.execute(query)
        res = cur.fetchall()
# https://stackoverflow.com/questions/5010042/mysql-get-column-name-or-alias-from-query
        col_names = [i[0] for i in cur.description]
        df = pd.DataFrame(res, columns=col_names)

        cur.close()

    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print('Something is wrong with your user name or password')
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print('Database does not exist')
        else:
            print(err)
    else:
        cnx.close()

    return df

### SQL Load

In [146]:
df_raw = run_sql('''select cdc_report_dt, 
    age_group,
    `Race and ethnicity (combined)`,
    sex,
	count(*) as reported_cases,
	sum(case current_status when "Laboratory-confirmed case" then 1 else 0 end) as confirmed_cases,
    sum(case hosp_yn when "Yes" then 1 else 0 end) as hosp,
    sum(case icu_yn when "Yes" then 1 else 0 end) as icu,
    sum(case medcond_yn when "Yes" then 1 else 0 end) as medcond,
    sum(case death_yn when "Yes" then 1 else 0 end) as deaths
from covid_19_us
where -- cdc_report_dt >= "2020-04-01" and 
age_group != "NA"
and age_group != "Unknown"
and `Race and ethnicity (combined)` != "NA"
and `Race and ethnicity (combined)` != "Unknown"
and `Race and ethnicity (combined)` != "Native Hawaiian/Other Pacific Islander, Non-Hispanic" -- 0.2% of cases
-- and `Race and ethnicity (combined)` != "American Indian/Alaska Native, Non-Hispanic	" -- 0.7% of cases
and (sex = "Male" or sex = "Female")
group by cdc_report_dt, `Race and ethnicity (combined)`, age_group, sex;''')

### Basic Clean and Save

In [147]:
df_raw.rename(columns={'cdc_report_dt':'date', 
                       'Race and ethnicity (combined)':'race_ethnicity',
                       'confirmed_cases':'cases'}, inplace=True)

df_us = df_raw.drop(df_raw[df_raw.deaths == 0].index)

cutoff_date = pd.to_datetime('2020-04-01', format='%Y-%m-%d', errors='coerce')
# drop rows ref:
# https://stackoverflow.com/questions/13851535/delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression-involving
df_us.drop(df_us[df_us.date < cutoff_date].index, inplace=True)

display(df_us)

,date,age_group,race_ethnicity,sex,reported_cases,cases,hosp,icu,medcond,deaths
2827,2020-04-01,20 - 29 Years,"American Indian/Alaska Native, Non-Hispanic",Male,3,3,1,0,0,1
2833,2020-04-01,50 - 59 Years,"American Indian/Alaska Native, Non-Hispanic",Male,4,4,1,0,0,2
2844,2020-04-01,30 - 39 Years,"Asian, Non-Hispanic",Male,7,7,1,0,1,1
2846,2020-04-01,40 - 49 Years,"Asian, Non-Hispanic",Male,21,21,7,2,5,2
2847,2020-04-01,50 - 59 Years,"Asian, Non-Hispanic",Female,28,28,10,4,10,3
...,...,...,...,...,...,...,...,...,...,...
24130,2020-10-16,60 - 69 Years,"White, Non-Hispanic",Male,1221,1147,114,9,225,24
24131,2020-10-16,70 - 79 Years,"White, Non-Hispanic",Female,779,731,96,3,170,24
24132,2020-10-16,70 - 79 Years,"White, Non-Hispanic",Male,690,649,120,7,149,32
24133,2020-10-16,80+ Years,"White, Non-Hispanic",Female,646,582,123,7,176,82


In [61]:
df_us.to_csv("us_age_race_sex.csv", index=False)

### Advanced Processing

In [148]:
df_i = df_raw.set_index(['date','age_group','race_ethnicity','sex'])

df_1wk = df_raw[['date','age_group','race_ethnicity','sex','deaths']].copy()

# Subtract days ref:
# https://stackoverflow.com/questions/20480897/pandas-add-one-day-to-column
df_1wk['date'] = df_1wk.date - pd.DateOffset(7)

cutoff_date = pd.to_datetime('2020-04-01', format='%Y-%m-%d', errors='coerce')
df_1wk.drop(df_1wk[df_1wk.date < cutoff_date].index, inplace=True)

df_1wk.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)
df_1wk = df_i.join(df_1wk, lsuffix='_1wk').dropna()

df_1wk.reset_index(inplace=True)
# df_1wk.drop('date', inplace=True)
df_1wk.drop(['date','deaths','reported_cases'], axis=1, inplace=True)
df_1wk.drop(df_1wk[df_1wk.deaths_1wk == 0].index, inplace=True)

display(df_1wk)

,age_group,race_ethnicity,sex,cases,hosp,icu,medcond,deaths_1wk
2,20 - 29 Years,"American Indian/Alaska Native, Non-Hispanic",Male,3,1,0,0,1
8,50 - 59 Years,"American Indian/Alaska Native, Non-Hispanic",Male,4,1,0,0,2
19,30 - 39 Years,"Asian, Non-Hispanic",Male,7,1,0,1,1
21,40 - 49 Years,"Asian, Non-Hispanic",Male,21,7,2,5,2
22,50 - 59 Years,"Asian, Non-Hispanic",Female,28,10,4,10,3
...,...,...,...,...,...,...,...,...
20450,60 - 69 Years,"White, Non-Hispanic",Male,626,63,6,120,10
20451,70 - 79 Years,"White, Non-Hispanic",Female,404,61,3,95,15
20452,70 - 79 Years,"White, Non-Hispanic",Male,433,88,11,108,19
20453,80+ Years,"White, Non-Hispanic",Female,353,69,7,106,44


In [62]:
df_1wk.to_csv("us_1week_delay.csv", index=False)

In [89]:
df_2wk = df_raw[['date','age_group','race_ethnicity','sex','deaths']].copy()

df_2wk['date'] = df_2wk.date - pd.DateOffset(14)

cutoff_date = pd.to_datetime('2020-04-01', format='%Y-%m-%d', errors='coerce')
df_2wk.drop(df_2wk[df_2wk.date < cutoff_date].index, inplace=True)

df_2wk.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)

df_2wk = df_i.join(df_2wk, rsuffix='_2wk').dropna()

df_2wk.reset_index(inplace=True)
df_2wk.drop(['date','deaths','reported_cases'], axis=1, inplace=True)
df_2wk.drop(df_2wk[df_2wk.deaths_2wk == 0].index, inplace=True)

display(df_2wk)

,age_group,race_ethnicity,sex,cases,hosp,icu,medcond,deaths_2wk
10,60 - 69 Years,"American Indian/Alaska Native, Non-Hispanic",Male,1,0,0,0,2
11,70 - 79 Years,"American Indian/Alaska Native, Non-Hispanic",Female,3,1,0,0,1
12,70 - 79 Years,"American Indian/Alaska Native, Non-Hispanic",Male,1,1,0,0,2
18,30 - 39 Years,"Asian, Non-Hispanic",Male,7,1,0,1,1
19,40 - 49 Years,"Asian, Non-Hispanic",Female,22,2,0,0,1
...,...,...,...,...,...,...,...,...
21937,60 - 69 Years,"White, Non-Hispanic",Male,567,52,6,96,24
21938,70 - 79 Years,"White, Non-Hispanic",Female,355,59,8,82,24
21939,70 - 79 Years,"White, Non-Hispanic",Male,389,86,12,98,32
21940,80+ Years,"White, Non-Hispanic",Female,262,68,4,65,82


In [63]:
df_2wk.to_csv("us_2week_delay.csv", index=False)

#### Rolling Average

In [149]:
df_roll = df_raw[['date','age_group','race_ethnicity','sex','deaths']].copy()

df_roll['date'] = df_roll.date - pd.DateOffset(14)

df_roll.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)

df_roll = df_roll.groupby(level=[1,2,3], as_index=False, dropna=True).rolling(14)['deaths'].mean().reset_index(level=[0,1,2], drop=True)

df_roll = df_i.join(df_roll, rsuffix='_roll').dropna()

df_roll.reset_index(inplace=True)
cutoff_date = pd.to_datetime('2020-04-01', format='%Y-%m-%d', errors='coerce')
df_roll.drop(df_roll[df_roll.date < cutoff_date].index, inplace=True)
df_roll.drop(['date','deaths','reported_cases'], axis=1, inplace=True)
df_roll.drop(df_roll[df_roll.deaths_roll == 0].index, inplace=True)

display(df_roll)

,age_group,race_ethnicity,sex,cases,hosp,icu,medcond,deaths_roll
2474,20 - 29 Years,"American Indian/Alaska Native, Non-Hispanic",Male,3,1,0,0,0.142857
2475,30 - 39 Years,"American Indian/Alaska Native, Non-Hispanic",Female,2,0,0,0,0.071429
2476,30 - 39 Years,"American Indian/Alaska Native, Non-Hispanic",Male,1,1,0,0,0.142857
2477,40 - 49 Years,"American Indian/Alaska Native, Non-Hispanic",Female,4,1,0,0,0.214286
2478,40 - 49 Years,"American Indian/Alaska Native, Non-Hispanic",Male,2,0,0,0,0.142857
...,...,...,...,...,...,...,...,...
22183,60 - 69 Years,"White, Non-Hispanic",Male,567,52,6,96,13.071429
22184,70 - 79 Years,"White, Non-Hispanic",Female,355,59,8,82,18.357143
22185,70 - 79 Years,"White, Non-Hispanic",Male,389,86,12,98,25.500000
22186,80+ Years,"White, Non-Hispanic",Female,262,68,4,65,55.214286


In [150]:
df_roll.to_csv("us_rolling.csv", index=False)

### Sanity Check

In [70]:
run_sql('''select `Race and ethnicity (combined)`, count(*) from covid_19_us
group by `Race and ethnicity (combined)`;''')

,Race and ethnicity (combined),count(*)
0,"American Indian/Alaska Native, Non-Hispanic",40991
1,"Asian, Non-Hispanic",101276
2,"Black, Non-Hispanic",583662
3,Hispanic/Latino,925883
4,"Multiple/Other, Non-Hispanic",168118
5,NA,10
6,"Native Hawaiian/Other Pacific Islander, Non-Hi...",12024
7,Unknown,2344715
8,"White, Non-Hispanic",1583387


### Scratch Code

In [115]:
df_roll2 = df_raw[['date','age_group','race_ethnicity','sex','deaths']].copy()

df_roll2['date'] = df_roll2.date - pd.DateOffset(12)

df_roll2.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)

df_roll2 = df_roll2.groupby(level=[1,2,3], as_index=False, dropna=False).rolling(7)['deaths'].mean()
df_roll2 = df_roll2.reset_index(level=[0,1,2], drop=True)

df_roll2 = df_i.join(df_roll2, rsuffix='_roll').dropna()

# display(df_roll2)

reported_cases cases  \
date       age_group     race_ethnicity      sex                            
2020-01-20 40 - 49 Years Black, Non-Hispanic Male                 1     1   
2020-01-21 50 - 59 Years White, Non-Hispanic Male                 1     1   
2020-01-25 50 - 59 Years White, Non-Hispanic Female               1     1   
2020-01-27 50 - 59 Years White, Non-Hispanic Female               1     1   
2020-01-31 50 - 59 Years White, Non-Hispanic Male                 1     1   
...                                                             ...   ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male               507   474   
           70 - 79 Years White, Non-Hispanic Female             334   307   
                                             Male               314   291   
           80+ Years     White, Non-Hispanic Female             230   213   
                                             Male               158   152   

                                                    hosp icu medcond deaths  \
date       age_group     race_ethnicity      sex                              
2020-01-20 40 - 49 Years Black, Non-Hispanic Male      1   1       1      0   
2020-01-21 50 - 59 Years White, Non-Hispanic Male      0   0       0      0   
2020-01-25 50 - 59 Years White, Non-Hispanic Female    0   0       0      0   
2020-01-27 50 - 59 Years White, Non-Hispanic Female    1   0       1      0   
2020-01-31 50 - 59 Years White, Non-Hispanic Male      1   0       0      0   
...                                                  ...  ..     ...    ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male     52   5     105      7   
           70 - 79 Years White, Non-Hispanic Female   44   5      97     11   
                                             Male     54   8      76      8   
           80+ Years     White, Non-Hispanic Female   50   6      68     28   
                                             Male     47  10      36     31   

                                                     deaths_roll  
date       age_group     race_ethnicity      sex                  
2020-01-20 40 - 49 Years Black, Non-Hispanic Male           0.00  
2020-01-21 50 - 59 Years White, Non-Hispanic Male           0.00  
2020-01-25 50 - 59 Years White, Non-Hispanic Female         0.00  
2020-01-27 50 - 59 Years White, Non-Hispanic Female         0.00  
2020-01-31 50 - 59 Years White, Non-Hispanic Male           0.00  
...                                                          ...  
2020-10-04 60 - 69 Years White, Non-Hispanic Male          16.50  
           70 - 79 Years White, Non-Hispanic Female        23.25  
                                             Male          34.75  
           80+ Years     White, Non-Hispanic Female        65.00  
                                             Male          67.00  

[24873 rows x 7 columns]

In [116]:
df_roll2a = df_raw[['date','age_group','race_ethnicity','sex','cases']].copy()
# df_roll2a.drop(['reported_cases'], axis=1, inplace=True)

df_roll2a['date'] = df_roll2a.date - pd.DateOffset(0)

df_roll2a.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)

df_roll2a = df_roll2a.groupby(level=[1,2,3], as_index=False, dropna=True).rolling(7)['cases'].mean()
df_roll2a = df_roll2a.reset_index(level=[0,1,2], drop=True)

df_roll2 = df_roll2.join(df_roll2a, rsuffix='_roll').dropna()

# display(df_roll2)

reported_cases cases  \
date       age_group     race_ethnicity      sex                            
2020-02-04 50 - 59 Years White, Non-Hispanic Male                 1     0   
2020-02-10 50 - 59 Years White, Non-Hispanic Male                 1     0   
           60 - 69 Years White, Non-Hispanic Female               1     1   
2020-02-11 40 - 49 Years White, Non-Hispanic Female               1     0   
           60 - 69 Years White, Non-Hispanic Female               2     1   
...                                                             ...   ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male               507   474   
           70 - 79 Years White, Non-Hispanic Female             334   307   
                                             Male               314   291   
           80+ Years     White, Non-Hispanic Female             230   213   
                                             Male               158   152   

                                                    hosp icu medcond deaths  \
date       age_group     race_ethnicity      sex                              
2020-02-04 50 - 59 Years White, Non-Hispanic Male      0   0       0      0   
2020-02-10 50 - 59 Years White, Non-Hispanic Male      0   0       0      0   
           60 - 69 Years White, Non-Hispanic Female    0   0       0      0   
2020-02-11 40 - 49 Years White, Non-Hispanic Female    0   0       0      0   
           60 - 69 Years White, Non-Hispanic Female    0   0       0      0   
...                                                  ...  ..     ...    ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male     52   5     105      7   
           70 - 79 Years White, Non-Hispanic Female   44   5      97     11   
                                             Male     54   8      76      8   
           80+ Years     White, Non-Hispanic Female   50   6      68     28   
                                             Male     47  10      36     31   

                                                     deaths_roll  cases_roll  
date       age_group     race_ethnicity      sex                              
2020-02-04 50 - 59 Years White, Non-Hispanic Male           0.00    1.000000  
2020-02-10 50 - 59 Years White, Non-Hispanic Male           0.00    0.571429  
           60 - 69 Years White, Non-Hispanic Female         0.25    0.714286  
2020-02-11 40 - 49 Years White, Non-Hispanic Female         0.00    0.285714  
           60 - 69 Years White, Non-Hispanic Female         0.25    0.714286  
...                                                          ...         ...  
2020-10-04 60 - 69 Years White, Non-Hispanic Male          16.50  751.428571  
           70 - 79 Years White, Non-Hispanic Female        23.25  489.714286  
                                             Male          34.75  495.142857  
           80+ Years     White, Non-Hispanic Female        65.00  401.142857  
                                             Male          67.00  275.142857  

[24387 rows x 8 columns]

In [117]:
df_roll2a = df_raw[['date','age_group','race_ethnicity','sex','hosp']].copy()

df_roll2a['date'] = df_roll2a.date - pd.DateOffset(7)

df_roll2a.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)

df_roll2a = df_roll2a.groupby(level=[1,2,3], as_index=False, dropna=True).rolling(4)['hosp'].mean()
df_roll2a = df_roll2a.reset_index(level=[0,1,2], drop=True)

df_roll2 = df_roll2.join(df_roll2a, rsuffix='_roll').dropna()

# display(df_roll2)

reported_cases cases  \
date       age_group     race_ethnicity      sex                            
2020-02-10 60 - 69 Years White, Non-Hispanic Female               1     1   
2020-02-12 50 - 59 Years White, Non-Hispanic Male                 1     0   
           60 - 69 Years White, Non-Hispanic Female               1     1   
2020-02-14 60 - 69 Years White, Non-Hispanic Male                 1     1   
2020-02-15 50 - 59 Years White, Non-Hispanic Male                 3     3   
...                                                             ...   ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male               507   474   
           70 - 79 Years White, Non-Hispanic Female             334   307   
                                             Male               314   291   
           80+ Years     White, Non-Hispanic Female             230   213   
                                             Male               158   152   

                                                    hosp icu medcond deaths  \
date       age_group     race_ethnicity      sex                              
2020-02-10 60 - 69 Years White, Non-Hispanic Female    0   0       0      0   
2020-02-12 50 - 59 Years White, Non-Hispanic Male      0   0       1      0   
           60 - 69 Years White, Non-Hispanic Female    0   0       1      0   
2020-02-14 60 - 69 Years White, Non-Hispanic Male      0   0       0      0   
2020-02-15 50 - 59 Years White, Non-Hispanic Male      0   0       1      0   
...                                                  ...  ..     ...    ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male     52   5     105      7   
           70 - 79 Years White, Non-Hispanic Female   44   5      97     11   
                                             Male     54   8      76      8   
           80+ Years     White, Non-Hispanic Female   50   6      68     28   
                                             Male     47  10      36     31   

                                                     deaths_roll  cases_roll  \
date       age_group     race_ethnicity      sex                               
2020-02-10 60 - 69 Years White, Non-Hispanic Female         0.25    0.714286   
2020-02-12 50 - 59 Years White, Non-Hispanic Male           0.00    0.428571   
           60 - 69 Years White, Non-Hispanic Female         0.25    0.714286   
2020-02-14 60 - 69 Years White, Non-Hispanic Male           0.25    0.714286   
2020-02-15 50 - 59 Years White, Non-Hispanic Male           0.00    0.714286   
...                                                          ...         ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male          16.50  751.428571   
           70 - 79 Years White, Non-Hispanic Female        23.25  489.714286   
                                             Male          34.75  495.142857   
           80+ Years     White, Non-Hispanic Female        65.00  401.142857   
                                             Male          67.00  275.142857   

                                                     hosp_roll  
date       age_group     race_ethnicity      sex                
2020-02-10 60 - 69 Years White, Non-Hispanic Female       0.75  
2020-02-12 50 - 59 Years White, Non-Hispanic Male         0.00  
           60 - 69 Years White, Non-Hispanic Female       0.75  
2020-02-14 60 - 69 Years White, Non-Hispanic Male         0.50  
2020-02-15 50 - 59 Years White, Non-Hispanic Male         0.00  
...                                                        ...  
2020-10-04 60 - 69 Years White, Non-Hispanic Male        61.50  
           70 - 79 Years White, Non-Hispanic Female      60.75  
                                             Male        79.25  
           80+ Years     White, Non-Hispanic Female      67.75  
                                             Male        65.00  

[24018 rows x 9 columns]

In [118]:
df_roll2a = df_raw[['date','age_group','race_ethnicity','sex','icu']].copy()

df_roll2a['date'] = df_roll2a.date - pd.DateOffset(10)

df_roll2a.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)

df_roll2a = df_roll2a.groupby(level=[1,2,3], as_index=False, dropna=True).rolling(4)['icu'].mean()
df_roll2a = df_roll2a.reset_index(level=[0,1,2], drop=True)

df_roll2 = df_roll2.join(df_roll2a, rsuffix='_roll').dropna()

# display(df_roll2)

reported_cases cases  \
date       age_group     race_ethnicity      sex                            
2020-02-10 60 - 69 Years White, Non-Hispanic Female               1     1   
2020-02-12 50 - 59 Years White, Non-Hispanic Male                 1     0   
           60 - 69 Years White, Non-Hispanic Female               1     1   
2020-02-15 50 - 59 Years White, Non-Hispanic Male                 3     3   
2020-02-16 40 - 49 Years White, Non-Hispanic Female               1     1   
...                                                             ...   ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male               507   474   
           70 - 79 Years White, Non-Hispanic Female             334   307   
                                             Male               314   291   
           80+ Years     White, Non-Hispanic Female             230   213   
                                             Male               158   152   

                                                    hosp icu medcond deaths  \
date       age_group     race_ethnicity      sex                              
2020-02-10 60 - 69 Years White, Non-Hispanic Female    0   0       0      0   
2020-02-12 50 - 59 Years White, Non-Hispanic Male      0   0       1      0   
           60 - 69 Years White, Non-Hispanic Female    0   0       1      0   
2020-02-15 50 - 59 Years White, Non-Hispanic Male      0   0       1      0   
2020-02-16 40 - 49 Years White, Non-Hispanic Female    0   0       1      0   
...                                                  ...  ..     ...    ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male     52   5     105      7   
           70 - 79 Years White, Non-Hispanic Female   44   5      97     11   
                                             Male     54   8      76      8   
           80+ Years     White, Non-Hispanic Female   50   6      68     28   
                                             Male     47  10      36     31   

                                                     deaths_roll  cases_roll  \
date       age_group     race_ethnicity      sex                               
2020-02-10 60 - 69 Years White, Non-Hispanic Female         0.25    0.714286   
2020-02-12 50 - 59 Years White, Non-Hispanic Male           0.00    0.428571   
           60 - 69 Years White, Non-Hispanic Female         0.25    0.714286   
2020-02-15 50 - 59 Years White, Non-Hispanic Male           0.00    0.714286   
2020-02-16 40 - 49 Years White, Non-Hispanic Female         0.00    0.428571   
...                                                          ...         ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male          16.50  751.428571   
           70 - 79 Years White, Non-Hispanic Female        23.25  489.714286   
                                             Male          34.75  495.142857   
           80+ Years     White, Non-Hispanic Female        65.00  401.142857   
                                             Male          67.00  275.142857   

                                                     hosp_roll  icu_roll  
date       age_group     race_ethnicity      sex                          
2020-02-10 60 - 69 Years White, Non-Hispanic Female       0.75      0.25  
2020-02-12 50 - 59 Years White, Non-Hispanic Male         0.00      0.00  
           60 - 69 Years White, Non-Hispanic Female       0.75      0.25  
2020-02-15 50 - 59 Years White, Non-Hispanic Male         0.00      0.25  
2020-02-16 40 - 49 Years White, Non-Hispanic Female       0.25      0.00  
...                                                        ...       ...  
2020-10-04 60 - 69 Years White, Non-Hispanic Male        61.50     10.50  
           70 - 79 Years White, Non-Hispanic Female      60.75      5.25  
                                             Male        79.25     13.00  
           80+ Years     White, Non-Hispanic Female      67.75      5.00  
                                             Male        65.00      9.00  

[23787

In [119]:
df_roll2a = df_raw[['date','age_group','race_ethnicity','sex','medcond']].copy()

df_roll2a['date'] = df_roll2a.date - pd.DateOffset(0)

df_roll2a.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)

df_roll2a = df_roll2a.groupby(level=[1,2,3], as_index=False, dropna=True).rolling(7)['medcond'].mean()
df_roll2a = df_roll2a.reset_index(level=[0,1,2], drop=True)

df_roll2 = df_roll2.join(df_roll2a, rsuffix='_roll').dropna()

# display(df_roll2)

reported_cases cases  \
date       age_group     race_ethnicity      sex                            
2020-02-10 60 - 69 Years White, Non-Hispanic Female               1     1   
2020-02-12 50 - 59 Years White, Non-Hispanic Male                 1     0   
           60 - 69 Years White, Non-Hispanic Female               1     1   
2020-02-15 50 - 59 Years White, Non-Hispanic Male                 3     3   
2020-02-16 40 - 49 Years White, Non-Hispanic Female               1     1   
...                                                             ...   ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male               507   474   
           70 - 79 Years White, Non-Hispanic Female             334   307   
                                             Male               314   291   
           80+ Years     White, Non-Hispanic Female             230   213   
                                             Male               158   152   

                                                    hosp icu medcond deaths  \
date       age_group     race_ethnicity      sex                              
2020-02-10 60 - 69 Years White, Non-Hispanic Female    0   0       0      0   
2020-02-12 50 - 59 Years White, Non-Hispanic Male      0   0       1      0   
           60 - 69 Years White, Non-Hispanic Female    0   0       1      0   
2020-02-15 50 - 59 Years White, Non-Hispanic Male      0   0       1      0   
2020-02-16 40 - 49 Years White, Non-Hispanic Female    0   0       1      0   
...                                                  ...  ..     ...    ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male     52   5     105      7   
           70 - 79 Years White, Non-Hispanic Female   44   5      97     11   
                                             Male     54   8      76      8   
           80+ Years     White, Non-Hispanic Female   50   6      68     28   
                                             Male     47  10      36     31   

                                                     deaths_roll  cases_roll  \
date       age_group     race_ethnicity      sex                               
2020-02-10 60 - 69 Years White, Non-Hispanic Female         0.25    0.714286   
2020-02-12 50 - 59 Years White, Non-Hispanic Male           0.00    0.428571   
           60 - 69 Years White, Non-Hispanic Female         0.25    0.714286   
2020-02-15 50 - 59 Years White, Non-Hispanic Male           0.00    0.714286   
2020-02-16 40 - 49 Years White, Non-Hispanic Female         0.00    0.428571   
...                                                          ...         ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male          16.50  751.428571   
           70 - 79 Years White, Non-Hispanic Female        23.25  489.714286   
                                             Male          34.75  495.142857   
           80+ Years     White, Non-Hispanic Female        65.00  401.142857   
                                             Male          67.00  275.142857   

                                                     hosp_roll  icu_roll  \
date       age_group     race_ethnicity      sex                           
2020-02-10 60 - 69 Years White, Non-Hispanic Female       0.75      0.25   
2020-02-12 50 - 59 Years White, Non-Hispanic Male         0.00      0.00   
           60 - 69 Years White, Non-Hispanic Female       0.75      0.25   
2020-02-15 50 - 59 Years White, Non-Hispanic Male         0.00      0.25   
2020-02-16 40 - 49 Years White, Non-Hispanic Female       0.25      0.00   
...                                                        ...       ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male        61.50     10.50   
           70 - 79 Years White, Non-Hispanic Female      60.75      5.25   
                                             Male        79.25     13.00   
           80+ Years     White, Non-Hispanic Female      67.75      5.00   
                                             Male        65.00      9.

In [120]:
df_roll2.reset_index(inplace=True)
cutoff_date = pd.to_datetime('2020-04-01', format='%Y-%m-%d', errors='coerce')
df_roll2.drop(df_roll2[df_roll2.date < cutoff_date].index, inplace=True)
df_roll2.drop(['date','deaths','reported_cases','cases','hosp','icu','medcond'], axis=1, inplace=True)
df_roll2.drop(df_roll2[df_roll2.deaths_roll == 0].index, inplace=True)
df_roll2.drop(df_roll2[df_roll2.cases_roll == 0].index, inplace=True)
df_roll2.drop(df_roll2[df_roll2.hosp_roll == 0].index, inplace=True)
df_roll2.drop(df_roll2[df_roll2.icu_roll == 0].index, inplace=True)
df_roll2.drop(df_roll2[df_roll2.medcond_roll == 0].index, inplace=True)

display(df_roll2)

,age_group,race_ethnicity,sex,deaths_roll,cases_roll,hosp_roll,icu_roll,medcond_roll
2187,60 - 69 Years,"American Indian/Alaska Native, Non-Hispanic",Male,1.25,2.571429,1.25,0.25,0.428571
2193,30 - 39 Years,"Asian, Non-Hispanic",Male,1.00,24.142857,6.50,1.00,3.714286
2194,40 - 49 Years,"Asian, Non-Hispanic",Female,0.50,26.000000,4.25,0.50,4.285714
2195,40 - 49 Years,"Asian, Non-Hispanic",Male,0.75,27.285714,11.50,1.00,7.428571
2196,50 - 59 Years,"Asian, Non-Hispanic",Female,3.00,32.142857,9.75,1.25,10.857143
...,...,...,...,...,...,...,...,...
23782,60 - 69 Years,"White, Non-Hispanic",Male,16.50,751.428571,61.50,10.50,138.714286
23783,70 - 79 Years,"White, Non-Hispanic",Female,23.25,489.714286,60.75,5.25,106.714286
23784,70 - 79 Years,"White, Non-Hispanic",Male,34.75,495.142857,79.25,13.00,114.000000
23785,80+ Years,"White, Non-Hispanic",Female,65.00,401.142857,67.75,5.00,90.142857


In [121]:
df_roll2.to_csv("us_roll_all.csv", index=False)

In [124]:
df_roll2 = df_raw[['date','age_group','race_ethnicity','sex','deaths']].copy()

df_roll2['date'] = df_roll2.date - pd.DateOffset(12)

df_roll2.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)

df_roll2 = df_roll2.groupby(level=[1,2,3], as_index=False, dropna=False).rolling(4)['deaths'].mean()
df_roll2 = df_roll2.reset_index(level=[0,1,2], drop=True)

df_roll2 = df_i.join(df_roll2, rsuffix='_roll').dropna()

display(df_roll2)

reported_cases cases  \
date       age_group     race_ethnicity      sex                            
2020-01-20 40 - 49 Years Black, Non-Hispanic Male                 1     1   
2020-01-21 50 - 59 Years White, Non-Hispanic Male                 1     1   
2020-01-25 50 - 59 Years White, Non-Hispanic Female               1     1   
2020-01-27 50 - 59 Years White, Non-Hispanic Female               1     1   
2020-01-31 50 - 59 Years White, Non-Hispanic Male                 1     1   
...                                                             ...   ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male               507   474   
           70 - 79 Years White, Non-Hispanic Female             334   307   
                                             Male               314   291   
           80+ Years     White, Non-Hispanic Female             230   213   
                                             Male               158   152   

                                                    hosp icu medcond deaths  \
date       age_group     race_ethnicity      sex                              
2020-01-20 40 - 49 Years Black, Non-Hispanic Male      1   1       1      0   
2020-01-21 50 - 59 Years White, Non-Hispanic Male      0   0       0      0   
2020-01-25 50 - 59 Years White, Non-Hispanic Female    0   0       0      0   
2020-01-27 50 - 59 Years White, Non-Hispanic Female    1   0       1      0   
2020-01-31 50 - 59 Years White, Non-Hispanic Male      1   0       0      0   
...                                                  ...  ..     ...    ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male     52   5     105      7   
           70 - 79 Years White, Non-Hispanic Female   44   5      97     11   
                                             Male     54   8      76      8   
           80+ Years     White, Non-Hispanic Female   50   6      68     28   
                                             Male     47  10      36     31   

                                                     deaths_roll  
date       age_group     race_ethnicity      sex                  
2020-01-20 40 - 49 Years Black, Non-Hispanic Male           0.00  
2020-01-21 50 - 59 Years White, Non-Hispanic Male           0.00  
2020-01-25 50 - 59 Years White, Non-Hispanic Female         0.00  
2020-01-27 50 - 59 Years White, Non-Hispanic Female         0.00  
2020-01-31 50 - 59 Years White, Non-Hispanic Male           0.00  
...                                                          ...  
2020-10-04 60 - 69 Years White, Non-Hispanic Male          16.50  
           70 - 79 Years White, Non-Hispanic Female        23.25  
                                             Male          34.75  
           80+ Years     White, Non-Hispanic Female        65.00  
                                             Male          67.00  

[24873 rows x 7 columns]

In [125]:
df_roll2a = df_raw[['date','age_group','race_ethnicity','sex','hosp']].copy()
df_roll2a['date'] = df_roll2a.date - pd.DateOffset(7)
df_roll2a.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)
df_roll2 = df_roll2.join(df_roll2a, rsuffix='_off7').dropna()

# display(df_roll2)

df_roll2a = df_raw[['date','age_group','race_ethnicity','sex','icu']].copy()
df_roll2a['date'] = df_roll2a.date - pd.DateOffset(10)
df_roll2a.set_index(['date','age_group','race_ethnicity','sex'], inplace=True)
df_roll2 = df_roll2.join(df_roll2a, rsuffix='_off10').dropna()

# display(df_roll2)

reported_cases cases  \
date       age_group     race_ethnicity      sex                            
2020-01-27 50 - 59 Years White, Non-Hispanic Female               1     1   
2020-02-02 40 - 49 Years White, Non-Hispanic Female               1     1   
           60 - 69 Years White, Non-Hispanic Female               1     0   
2020-02-09 60 - 69 Years White, Non-Hispanic Female               1     1   
2020-02-10 60 - 69 Years White, Non-Hispanic Female               1     1   
...                                                             ...   ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male               507   474   
           70 - 79 Years White, Non-Hispanic Female             334   307   
                                             Male               314   291   
           80+ Years     White, Non-Hispanic Female             230   213   
                                             Male               158   152   

                                                    hosp icu medcond deaths  \
date       age_group     race_ethnicity      sex                              
2020-01-27 50 - 59 Years White, Non-Hispanic Female    1   0       1      0   
2020-02-02 40 - 49 Years White, Non-Hispanic Female    0   0       0      0   
           60 - 69 Years White, Non-Hispanic Female    0   0       0      0   
2020-02-09 60 - 69 Years White, Non-Hispanic Female    0   0       0      0   
2020-02-10 60 - 69 Years White, Non-Hispanic Female    0   0       0      0   
...                                                  ...  ..     ...    ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male     52   5     105      7   
           70 - 79 Years White, Non-Hispanic Female   44   5      97     11   
                                             Male     54   8      76      8   
           80+ Years     White, Non-Hispanic Female   50   6      68     28   
                                             Male     47  10      36     31   

                                                     deaths_roll hosp_off7  \
date       age_group     race_ethnicity      sex                             
2020-01-27 50 - 59 Years White, Non-Hispanic Female         0.00         0   
2020-02-02 40 - 49 Years White, Non-Hispanic Female         0.00         0   
           60 - 69 Years White, Non-Hispanic Female         0.00         0   
2020-02-09 60 - 69 Years White, Non-Hispanic Female         0.50         1   
2020-02-10 60 - 69 Years White, Non-Hispanic Female         0.25         1   
...                                                          ...       ...   
2020-10-04 60 - 69 Years White, Non-Hispanic Male          16.50        60   
           70 - 79 Years White, Non-Hispanic Female        23.25        58   
                                             Male          34.75        72   
           80+ Years     White, Non-Hispanic Female        65.00        61   
                                             Male          67.00        64   

                                                    icu_off10  
date       age_group     race_ethnicity      sex               
2020-01-27 50 - 59 Years White, Non-Hispanic Female         0  
2020-02-02 40 - 49 Years White, Non-Hispanic Female         0  
           60 - 69 Years White, Non-Hispanic Female         0  
2020-02-09 60 - 69 Years White, Non-Hispanic Female         0  
2020-02-10 60 - 69 Years White, Non-Hispanic Female         0  
...                                                       ...  
2020-10-04 60 - 69 Years White, Non-Hispanic Male           6  
           70 - 79 Years White, Non-Hispanic Female         1  
                                             Male          11  
           80+ Years     White, Non-Hispanic Female         6  
                                             Male           7  

[24124 rows x 9 columns]

In [126]:
df_roll2.reset_index(inplace=True)
cutoff_date = pd.to_datetime('2020-04-01', format='%Y-%m-%d', errors='coerce')
df_roll2.drop(df_roll2[df_roll2.date < cutoff_date].index, inplace=True)
df_roll2.drop(['date','deaths','reported_cases','hosp','icu'], axis=1, inplace=True)
df_roll2.drop(df_roll2[df_roll2.deaths_roll == 0].index, inplace=True)

display(df_roll2)

,age_group,race_ethnicity,sex,cases,medcond,deaths_roll,hosp_off7,icu_off10
2514,30 - 39 Years,"American Indian/Alaska Native, Non-Hispanic",Male,1,0,0.25,0,0
2518,50 - 59 Years,"American Indian/Alaska Native, Non-Hispanic",Male,4,0,0.50,0,0
2519,60 - 69 Years,"American Indian/Alaska Native, Non-Hispanic",Female,1,0,0.25,3,0
2520,60 - 69 Years,"American Indian/Alaska Native, Non-Hispanic",Male,1,0,1.25,1,1
2521,70 - 79 Years,"American Indian/Alaska Native, Non-Hispanic",Female,3,0,0.50,1,0
...,...,...,...,...,...,...,...,...
24119,60 - 69 Years,"White, Non-Hispanic",Male,474,105,16.50,60,6
24120,70 - 79 Years,"White, Non-Hispanic",Female,307,97,23.25,58,1
24121,70 - 79 Years,"White, Non-Hispanic",Male,291,76,34.75,72,11
24122,80+ Years,"White, Non-Hispanic",Female,213,68,65.00,61,6


In [127]:
df_roll2.to_csv("us_roll_off.csv", index=False)

In [44]:
df_adv.reset_index()

,date,age_group,race_ethnicity,sex,reported_cases,confirmed_cases,hosp,icu,medcond,deaths,deaths_1wk,deaths_2wk,deaths_roll
0,2020-04-01,20 - 29 Years,"American Indian/Alaska Native, Non-Hispanic",Female,2,2,1,0,0,0,0,0,0.000000
1,2020-04-01,20 - 29 Years,"American Indian/Alaska Native, Non-Hispanic",Male,3,3,1,0,0,1,1,0,0.142857
2,2020-04-01,30 - 39 Years,"American Indian/Alaska Native, Non-Hispanic",Female,2,2,0,0,0,0,0,0,0.000000
3,2020-04-01,30 - 39 Years,"American Indian/Alaska Native, Non-Hispanic",Male,1,1,1,0,0,0,0,0,0.142857
4,2020-04-01,40 - 49 Years,"American Indian/Alaska Native, Non-Hispanic",Female,4,4,1,0,0,0,1,0,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21383,2020-10-02,60 - 69 Years,"White, Non-Hispanic",Male,620,567,52,6,96,13,10,24,11.714286
21384,2020-10-02,70 - 79 Years,"White, Non-Hispanic",Female,388,355,59,8,82,15,15,24,17.571429
21385,2020-10-02,70 - 79 Years,"White, Non-Hispanic",Male,415,389,86,12,98,22,19,32,21.857143
21386,2020-10-02,80+ Years,"White, Non-Hispanic",Female,278,262,68,4,65,48,44,82,51.000000


In [ ]:
df_adv.to_csv("us_covid_adv.csv")

In [4]:
# age = run_sql('''select cdc_report_dt, age_group, count(*) from covid_19_us group by cdc_report_dt, age_group;''')

In [5]:
# age.columns = ['date', 'age', 'cases']
# age_ind = age.set_index(['date', 'age'])
# display(age_ind.unstack())


In [6]:
# data_frame = run_sql("""
# select med.*, onset from
# 	(select cdc_report_dt, 
# 	count(*) as total_cases,
# 	sum(case current_status when "Laboratory-confirmed case" then 1 else 0 end) as confirmed_cases,
#     sum(case sex when "Male" then 1 else 0 end) as male, 
#     sum(case age_group when "0 - 9 Years" then 1
# 		when "10 - 19 Years" then 1
#         when "20 - 29 Years" then 1
# 		else 0 end) as age0_29, 
#     sum(case age_group when "30 - 39 Years" then 1 else 0 end) as age30_39, 
# 	sum(case age_group when "40 - 49 Years" then 1 else 0 end) as age40_49, 
#     sum(case age_group when "50 - 59 Years" then 1 else 0 end) as age50_59, 
#     sum(case age_group when "60 - 69 Years" then 1 else 0 end) as age60_69, 
#     sum(case age_group when "70 - 79 Years" then 1 else 0 end) as age70_79, 
#     sum(case age_group when "80+ Years" then 1 else 0 end) as age80_up,
#     sum(case `Race and ethnicity (combined)` when "Asian, Non-Hispanic" then 1 else 0 end) as r_asian,
#     sum(case `Race and ethnicity (combined)` when "Multiple/Other, Non-Hispanic" then 1 else 0 end) as r_mult,
#     sum(case `Race and ethnicity (combined)` when "Black, Non-Hispanic" then 1 else 0 end) as r_black, 
#     sum(case `Race and ethnicity (combined)` when "Hispanic/Latino" then 1 else 0 end) as r_hisp,
#     sum(case hosp_yn when "Yes" then 1 else 0 end) as hosp,
#     sum(case icu_yn when "Yes" then 1 else 0 end) as icu,
#     sum(case medcond_yn when "Yes" then 1 else 0 end) as medcond,
#     sum(case death_yn when "Yes" then 1 else 0 end) as deaths
# from covid_19_us
# group by cdc_report_dt) as med
# join
# 	(select onset_dt, count(*) as onset from covid_19_us 
# 	where onset_dt != "0000-00-00"
# 	group by onset_dt) as onset
# on cdc_report_dt = onset_dt;""")

In [7]:
# data_frame.head()
# # display(data_frame)

# data_frame.columns = ["date", "total", "conf", "male", "age0_29", "age30_39", "age40_49", 
#                       "age50_59", "age60_69", "age70_79", "age80_up", "r_asian", 
#                       "r_mult", "r_black", "r_hisp", "hosp", "icu", "medcond", "deaths", "onset"]
# df = data_frame.set_index("date")
# display(df)

# df.to_csv("us_totals_category.csv")

In [23]:
df2.loc[(df2["deaths"] == 0) & (df2["reported_cases"] > 10)].sort_values(by="reported_cases", ascending=False)
df2.loc[(df2["deaths"] == 0) & (df2["reported_cases"] > 100)].sort_values(by="reported_cases", ascending=False)
# df2.loc[(df2["deaths"] == 0) & (df2["reported_cases"] > 1000)].sort_values(by="reported_cases", ascending=False)

,cdc_report_dt,age_group,Race and ethnicity (combined),sex,reported_cases,confirmed_cases,hosp,icu,medcond,deaths
19080,2020-09-05,20 - 29 Years,"White, Non-Hispanic",Female,4104,3903,14,0,51,0
19081,2020-09-05,20 - 29 Years,"White, Non-Hispanic",Male,3564,3381,14,0,56,0
22234,2020-10-01,20 - 29 Years,"White, Non-Hispanic",Male,2834,2669,11,0,107,0
12449,2020-07-13,20 - 29 Years,"White, Non-Hispanic",Male,2512,2491,22,2,188,0
19078,2020-09-05,10 - 19 Years,"White, Non-Hispanic",Female,2476,2330,5,0,40,0
...,...,...,...,...,...,...,...,...,...,...
13803,2020-07-24,0 - 9 Years,"White, Non-Hispanic",Female,101,93,1,0,2,0
22286,2020-10-02,10 - 19 Years,"Black, Non-Hispanic",Male,101,85,3,0,10,0
4453,2020-05-09,10 - 19 Years,Hispanic/Latino,Female,101,99,5,1,9,0
14392,2020-07-29,20 - 29 Years,"Multiple/Other, Non-Hispanic",Male,101,99,2,0,2,0


In [21]:
df2.loc[(df2["deaths"] == 0) 
        & (df2["reported_cases"] > 100)
        & (df2["age_group"] != "0 - 9 Years")
        & (df2["age_group"] != "10 - 19 Years")
        & (df2["age_group"] != "20 - 29 Years")
        & (df2["age_group"] != "30 - 39 Years")].sort_values(by="reported_cases", ascending=False)

,cdc_report_dt,age_group,Race and ethnicity (combined),sex,reported_cases,confirmed_cases,hosp,icu,medcond,deaths
23712,2020-10-13,40 - 49 Years,"White, Non-Hispanic",Male,1569,1461,55,6,263,0
23467,2020-10-11,50 - 59 Years,"White, Non-Hispanic",Female,1409,1288,32,1,103,0
24073,2020-10-16,40 - 49 Years,"White, Non-Hispanic",Female,1388,1280,23,1,187,0
17131,2020-08-20,40 - 49 Years,"White, Non-Hispanic",Female,1270,1234,27,3,198,0
23589,2020-10-12,50 - 59 Years,"White, Non-Hispanic",Female,1111,1031,37,3,132,0
...,...,...,...,...,...,...,...,...,...,...
23921,2020-10-15,40 - 49 Years,"Multiple/Other, Non-Hispanic",Male,102,97,3,0,2,0
16977,2020-08-19,50 - 59 Years,"Multiple/Other, Non-Hispanic",Female,102,101,3,0,3,0
20832,2020-09-20,50 - 59 Years,"Black, Non-Hispanic",Male,102,101,7,0,9,0
8027,2020-06-07,40 - 49 Years,"White, Non-Hispanic",Female,102,93,3,0,23,0


In [22]:
df3 = run_sql('''select cdc_report_dt, 
    age_group,
    `Race and ethnicity (combined)`,
    sex,
	count(*) as reported_cases,
	sum(case current_status when "Laboratory-confirmed case" then 1 else 0 end) as confirmed_cases,
    sum(case hosp_yn when "Yes" then 1 else 0 end) as hosp,
    sum(case icu_yn when "Yes" then 1 else 0 end) as icu,
    sum(case medcond_yn when "Yes" then 1 else 0 end) as medcond,
    sum(case death_yn when "Yes" then 1 else 0 end) as deaths
from covid_19_us
where -- cdc_report_dt >= "2020-04-01" 
age_group != "NA"
and age_group != "Unknown"
and `Race and ethnicity (combined)` != "NA"
and `Race and ethnicity (combined)` != "Unknown"
and (sex = "Male" or sex = "Female")
group by cdc_report_dt, `Race and ethnicity (combined)`, age_group, sex;''')

In [24]:
# df3["d_1week"] = df3["deaths"]
df3_i = df3.set_index(['cdc_report_dt','age_group','Race and ethnicity (combined)','sex'])